In [1]:
import random

In [ ]:
def combinacoes_de_dados(lista):
    combinacoes = set()

    combinacoes.add((lista[0] + lista[1],lista[2] + lista[3]))
    combinacoes.add((lista[0] + lista[2],lista[1] + lista[3]))
    combinacoes.add((lista[0] + lista[3],lista[1] + lista[2]))

    return list(combinacoes)


In [2]:
class Column():
    def __init__(self,length):
        self.length=length
        self.position={1:0,2:0}
        self.marker=0
        self.blocked=False

    def advance_marker(self):
        self.marker+=1

    def record_advance(self,active_player):
        self.position[active_player] = self.marker
        self.marker_reset()

    def marker_reset(self):
        self.marker = 0

    def maker_start(self,active_player):
        self.marker = self.position[active_player] + 1

    def possui_marcador(self):
        return self.marker > 0

In [3]:
n_colunas = [2,4,6,8,10,12,10,8,6,4,2]
class Board():
    def __init__(self):
        self.columns={}
        for i in range(11):
            self.columns[i+2] = Column(n_colunas[i])
    
    def roll_dice(self):
        return combinacoes_de_dados([random.randrange(1, 7),random.randrange(1, 7),random.randrange(1, 7),random.randrange(1, 7)])

    def lista_colunas_com_marcadores(self):
        return list(filter( lambda coluna : self.columns[coluna].possui_marcador(), self.columns))

In [4]:
board=Board()

In [5]:
board.roll_dice()

[5, 2, 4, 1]

In [6]:
board.columns[2]

In [7]:
len(board.columns)

11

In [8]:
board.columns[2].length

2

In [9]:
roll = (random.randrange(1, 7),random.randrange(1, 7),random.randrange(1, 7),random.randrange(1, 7))
roll

(1, 1, 6, 2)